# Proof and Demo of ApEn and SampEn of a Markov Chain
#### By Katherine Wuestney  

The following notebook provides a demonstration of the mc_entropy module and the functions for calculating the true entropy rate, Approximate Entropy, and Sample Entropy of a Markov Process with known transition probabilities. Accompanying this demonstration is a proof of the formula for the true Sample Entropy of a Markov Chain.



*Note: In this demonstration, all discussion of indexing or elements at index i assumes a series or sequence indexed at 1 for the first element, as opposed to the Python indexing convention which starts at i=0. This means that some of the indexing used in the Python code blocks will not align completely with that used in the text. Therefore when the range $1 \leq i \leq N-m$ is given in the text below, this is equivalent to the range $0 \leq i \leq N-m-1$ in Python indexing.*

In [1]:
import sys
import numpy as np
from .mc_measures import mc_entropy as mce
from .mc_measures import gen_mc_transition as GMTP
sys.path.append(r'C:\Users\Wuestney\Documents\GitHub')
from discreteMSE.discreteMSE.entropy import sampen, apen

## Overview of mc_entropy.py
The module mc_entropy.py is part of the mc_measures package and all three functions in mc_entropy.py are designed to accept an instance of the GenMarkovTransitionProb (GMTP) class from the gen_mc_transition.py module.

There are three functions in mc_entropy.py: entropy_rate(), markov_apen(), and markov_sampen().


### mc_entropy.entropy_rate()
The entropy rate of a stationary, ergodic Markov Chain, $X$, with state space $\Omega$, steady state probability vector, $\pi$, and transition matrix $P$ is :
$$
\mathcal{H}(X) = -\sum_{j\in\Omega}\sum_{i\in\Omega}\pi_jP_{ij}log(P_{ij})
$$
where $\pi_j = Pr(X=j)$, the stationary probability of state j, and $P$ is a left stochastic matrix whose columns sum to 1 and $P_{ij}$ is the probability that the next state in the Markov Chain is $i$ given the present state is $j$, (i.e. $Pr(X_{n+1}=i | X_n=j)$) (see [1](#1), Theorem 4.2.4).  
  
The function entropy_rate() is identical to the entropy_rate class method of the GMTP class. It requires a square transition matrix as it uses eigendecomposition to obtain the steady state probability vector from the transition matrix.

### mc_entropy.markov_apen()
Pincus [2](#2) established that the Approximate Entropy of a Markov chain is equivalent to the same entropy rate formula of a Markov chain above when ($r < min(|\Omega_i - \Omega_j|, i \neq j, i \ and \ j \ state \ space \ values$). This is  the same condition when applying Approximate Entropy to any arbitrary discrete valued state space. Pincus predicts this to be true almost surely for any value of *m*. Because of this, markov_apen() behaves as a wrapper for entropy_rate() and performs the same function as entropy_rate().

   - Example. Take the Markov Chain with transition probabilities $P_{1, 3}=2/3$, $P_{2, 1}=1$, $P_{3, 2}=1$, $P_{3,3}=1/3$. *Note each column sums to 1*.
   - We compute the stationary probabilities to be $\pi_1=2/7$, $\pi_2=2/7$, and $\pi_3=3/7$.
   - Next we plug in the probabilities to the above equation: 
$$
\begin{align}
\mathcal{H}(X) = ApEn(X) &= - (2/7*0log(0) + 2/7*1log(1) + 2/7*0log(0) + 2/7*0log(0) + 2/7*0log(0) + 2/7*1log(1) + \\ 
& \ \ \ \ 3/7*2/3log(2/3) + 3/7*0log(0) + 3/7*1/3log(1/3)) \\
&= - (2/7log(2/3) + 1/7log(1/3)) \\
&= 0.273
\end{align}
$$

In [2]:
#load example markov chain used by Pincus [1] as GMTP class instance
fpath = GMTP.get_data_file_path(out_dir=r'mc_matrices', out_name=r'PincusEx.Order1Alph3.json')
pincus = GMTP.get_model(fpath)

In [3]:
#calc entropy rate of the Pincus MC model
pcs_er = mce.entropy_rate(pincus)
pcs_apen = mce.markov_apen(pincus)
print("Entropy rate of Pincus' MC model: ", pcs_er)
print("Expected ApEn of Pincus' MC model: ", pcs_apen)

Entropy rate of Pincus' MC model:  0.2727917864120626
Expected ApEn of Pincus' MC model:  0.2727917864120626


#### GMTP.eig_steadystate() algorithm
As mentioned above, the entropy_rate() function uses computes the steady state vector of the Markov chain using eigen decomposition. This process is performed by the GMTP.eig_steadystate() method and explained here:

1. Convert the square transition matrix to a numpy array using method GMTP.\_asarray() and store in local variable. *See test_gmtp notebook for explanation of the GMTP.\_asarray() method.*
2. Convert the transition frequencies to transition probabilities by dividing each entry by the sum of its column.
3. Compute the eigenvalues and eigenvectors associated with the transition matrix using np.linalg.eig(). The output of np.linalg.eig() is a 1d numpy array of eigenvalues and 2d numpy array of eigenvectors where the index of each eigenvalue corresponds to the column index of the eigenvectors.
4. Get the basis vector for the steady state probability vector by finding the index associated with the eigenvalue equal to 1 then getting the column vector at that index from the eigenvectors array. The eigenvector associated with the eigenvalue equal to 1.0 is a basis for the steady state probility vector of the transition probability matrix.
5. Normalize the basis vector to sum to 1 to get the steady state vector. 
6. Return the steady state vector and transition probability matrix as output.

In [4]:
#print stationary probabilities and transition matrix
pi, P = pincus.eig_steadystate()
print('Stationary probabilites: \n', pi)
print("Transition matrix (columns sum to 1): \n", P)

Stationary probabilites: 
 [0.28571429 0.28571429 0.42857143]
Transition matrix (columns sum to 1): 
 [[0.         0.         0.66666667]
 [1.         0.         0.        ]
 [0.         1.         0.33333333]]


See how the stationary probabilities computed from eigen decomposition match the ones given by Pincus

In [5]:
pi1 = 2/7
pi2 = 2/7
pi3 = 3/7
print("Stationary probabilities from Pincus: \n", np.array([pi1, pi2, pi3]))

Stationary probabilities from Pincus: 
 [0.28571429 0.28571429 0.42857143]


### mc_entropy.markov_sampen()
The expected Sample Entropy of a markov chain relies on the properties of the SampEn for discrete-valued data. Recall the basic definition of the parameter $SampEn(m, r) = \lim_{N\to\infty} -ln[A^m(r)/B^m(r)]$, where $B^m(r)$ is the probability that any two subsequences of $X$ will match for *m* points and $A^m(r)$ is the probability that any two subsequences in $X$ will match for *m+1* points. Again recall the alternative formulation of the parameter $SampEn(m, r) = \lim_{N\to\infty} -ln[A/B]$ where $B$ is the total number of *m*-length vector matches (excluding self matches) in the sample sequence $X$ and $A$ is the total number of *m+1*-length vector matches (excluding self matches) in $X$ [3](#3). Then, according to [3](#3) the quantity $A/B$ is the conditional probability that two subsequences in $X$ are within *r* of each other for *m+1* points given they are within *r* of each other for *m* points.  
$$
A/B = Pr(d[x_{m+1}(i), x_{m+1}(j)]\leq r \ |\  d[x_{m}(i),x_{m}(j)] \leq r)\ \forall i,j \in X
$$

### Proof of Sample Entropy of a Markov Chain
Consider the scenario with the kth-order Markov process $X$ with state space $S$ and transition probabilities, $P$, and steady state probabilities $\pi$ and where $m=k$. The steady state distribution of $X$ can be treated as a multinomial distribution where $\pi_i = Pr(X = S_i)$ and the expected count of a given state, $S_i$ in a sample sequence ${X_1, X_2, ..., X_{N}}$ is given by $N*\pi_i$. Recall in the original SampEn algorithm, for each *m*-length vector, $x_m(i)$, from $X, 1 \leq i \leq N-m$, the proportional frequency $B_i^m(r=0)$ is computed as:  
$$
B_i^m(r) = (N-m-1)^{-1}[number \  of x_m(j), (1\leq j \leq N-m, i\neq j)\  where\  d[x_m(i), x_m(j)]\leq r]
$$.  
In other words $B_i^m(r)$ is the proportion of the $N-m-1$ set of vectors excluding $x_m(i)$ which match $x_m(i)$. *Recall self-matches are excluded from the counts, thus the subtraction of 1*. In the discrete-valued case, where $x_m(i) = S_i; S_i \in S$, we can compute the expected frequency of $S_i$ in the $N-m-1$ set of $x_m(j)$ as $(N-m-1)*\pi_i$ and thus the expected value, as $N\to\infty$, of $B_i^m(r)$ is:
$$
\begin{align}
B_i^m(r=0)&=(N-m-1)^{-1}(N-m-1)\pi_i\\ 
&= \pi_i
\end{align}
$$

Recall that the quantity $B^m(r)$ from above is defined as $(N-m)^{-1}\sum_{i=1}^{N-m}B_i^m(r)$; the mean of the $B_i^m(r)$ values. Because only exact matches are included in the discrete-valued case, each instance of $S_i$ in the sample will contribute the same value of $B_i^m(r=0)=\pi_i$ to the sum for $B^m(r=0)$. We expect there to be about $(N-m)*\pi_i$ instances of $S_i$, each contributing the value $\pi_i$ to the sum and so substituting in the above formula we get  
$$
\begin{align}
B^m(r=0)&=(N-m)^{-1}\sum_{i \in S}(N-m)\pi_i\pi_i \\
&= \sum_{i \in S}\pi_i\pi_i
\end{align}
$$.

Similarly, we can compute the expected proportional frequency of *m+1*-length vectors matching $x_{m+1}(i)$ in the $N-m-1$ set of vectors excluding $x_{m+1}(i)$, where $x_{m+1}(i)$ is an instance of the joint distribution $(S_i, S_j)$. Because of the Markov Property, $Pr(X_{n,n+1} = (S_i,S_j))=Pr(X_n = S_i)*Pr(X_{n+1} = S_j | X_n = S_i)$. Using the steady state probability distribution and transition matrix we then can calculate the expected frequency of vectors matching $x_{m+1}(i)$ as $(N-m)\pi_iP_ij$. Therefore, we compute the quantity  
$$
\begin{align}
A_i^m(r=0)&=(N-m-1)^{-1}(N-m-1)\pi_iP_{ij} \\
&= \pi_iP_{ij}
\end{align}
$$   
From here, following the same logic as for $B^m(r=0)$ we can calculate  
$$
\begin{align}
A^m(r=0) & = (N-m)^{-1}\sum_{i\in S}\sum_{j \in S}(N-m)\pi_iP_{ij}\pi_iP_{ij}\\
& = \sum_{i\in S}\sum_{j \in S}\pi_iP_{ij}\pi_iP_{ij}
\end{align}
$$ 

From here we get the formula for the expected value of SampEn for a discrete-valued Markov Chain:  
$$
SampEn(m, r=0) = \lim_{N\to\infty} -ln\left(\frac{\sum_{i\in S}\sum_{j \in S}\pi_iP_{ij}\pi_iP_{ij}}{\sum_{i \in S}\pi_i\pi_i}\right)
$$

   - Example. Using the same Markov transition matrix example from Pincus and introduced above we calculate the expected SampEn of the Markov process as follows:  

$$
\begin{align}
SampEn(1, r=0) &= - ln \left(\frac{(2/7*0)^2 + (2/7*1)^2 + (2/7*0)^2 + (2/7*0)^2 + (2/7*0)^2 + (2/7*1)^2 + (3/7*2/3)^2 + (3/7*0)^2 + (3/7*1/3)^2}{(2/7)^2 + (2/7)^2 + (3/7)^2}\right)\\
&= -ln \left(\frac{(2/7)^2 + (2/7)^2 + (2/7)^2 + (1/7)^2}{(2/7)^2 + (2/7)^2 + (3/7)^2}\right) \\
&= -ln \left(13/17\right) \\
&=0.268
\end{align}
$$

In [6]:
pcs_sampen = mce.markov_sampen(pincus)
print("Expected SampEn of Pincus' MC model: ", pcs_sampen)

Expected SampEn of Pincus' MC model:  0.26826398659467926


## References
<a id='1'></a>
<div class="csl-entry">[1] Cover, T. M., &#38; Thomas, J. A. (2006). <i>Elements of Information Theory</i> (Second). John Wiley &#38; Sons, Inc.</div>
<a id='2'></a>
<div class="csl-entry">[2] Pincus, S. M. (1991). Approximate entropy as a measure of system complexity. <i>Proceedings of the National Academy of Sciences of the United States of America</i>, <i>88</i>(6), 2297–2301. https://doi.org/10.1073/pnas.88.6.2297</div>
<a id='3'></a>
<div class="csl-entry">[3] Richman, J. S., &#38; Moorman, J. R. (2000). Physiological time-series analysis using approximate entropy and sample entropy. <i>Americal Journal of Physiology Heart and Circulatory Physiology</i>, <i>278</i>, H2039–H2049.</div>